In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

from lxmls.transformers.model import GPT
from lxmls.transformers.utils import set_seed
from lxmls.transformers.bpe import BPETokenizer

In [ ]:
model_type = 'gpt2'
device = 'cuda'

In [ ]:
model = GPT.from_pretrained(model_type)

In [ ]:
# ship model to device and set to eval mode
model.to(device)
model.eval();

In [ ]:
def generate(prompt='', num_samples=10, tokens=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    tokenizer = BPETokenizer()
    if prompt == '':
        # to create unconditional samples...
        # manually create a tensor with only the special <|endoftext|> token
        # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
        x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
    else:
        x = tokenizer(prompt).to(device)
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=tokens, do_sample=do_sample, top_k=num_samples*5)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [ ]:
generate(prompt='Ramon Astudio, the', num_samples=10, tokens=50)